In [3]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers,optimizers
import numpy as np
import keras
from keras.models import Sequential,Model
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Deconvolution2D, Cropping2D
from keras.layers import Input, Add, Dropout, Permute, add

import copy
import math
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline



Using TensorFlow backend.


In [ ]:
from keras.callbacks import ReduceLROnPlateau, CSVLogger,EarlyStopping
from scipy.misc import toimage, imresize

In [4]:
def saliency_detection():
    vgg_16_model = VGG16(weights=None,include_top = False)
     
    #get weights from pretrained model on imagenet
    #vgg_16_model.load_weights('vgg_weights.h5')
    
    inputs = Input(shape = (224,224,3), name = "image_input")
    
    #create dummy layer
    output_vgg16_model = vgg_16_model(inputs)

    #Making the network fully convolutional 
    conv1_7x7 = Convolution2D(4096,kernel_size=(7,7),padding = "same",activation = "relu",name = "FullConv1")(output_vgg16_model)
    dropout_1 = Dropout(0.2)(conv1_7x7)
    conv2_1x1 = Convolution2D(4096,kernel_size=(1,1),padding = "same",activation = "relu",name = "FullConv2")(dropout_1)
    dropout_2 = Dropout(0.2)(conv2_1x1)
    
    conv3_1x1 = Convolution2D(1,kernel_size=(1,1),padding = "same",activation = "relu",name = "FullConv3")(dropout_2)
    deconv1_1x1 = Deconvolution2D(1,kernel_size=(1,1), strides= (32,32),padding = "valid",activation='softmax',name = "Deconv1")(conv3_1x1)
    
    return Model(inputs = inputs, outputs = deconv1_1x1)

In [5]:
model =  saliency_detection()

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
FullConv1 (Conv2D)           (None, 7, 7, 4096)        102764544 
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 4096)        0         
_________________________________________________________________
FullConv2 (Conv2D)           (None, 7, 7, 4096)        16781312  
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 7, 4096)        0         
_________________________________________________________________
FullConv3 (Conv2D)           (None, 7, 7, 1)           4097      
__________

In [ ]:
#Defining Callback functions which will be called by model during runtime when specified condition satisfies
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
csv_logger = CSVLogger('./vgg16_saliency.csv')
early_stopper = EarlyStopping(min_delta=0.001, patience=10)

In [ ]:
model.compile(loss = 'mean_squared_error',optimizer='adam',metrics = ['accuracy'])

In [ ]:
train_images = np.load('train_images_saliency.npy')
train_labels = np.load('train_labels_saliency.npy')

In [ ]:
data_augmentation = False

In [ ]:
if data_augmentation :
    print("-------------Using Data augmentation------------")
     # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    
    datagen.fit(x_train)
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,verbose=1,validation_data=(x_test,y_test),callbacks=[lr_reducer, early_stopper, csv_logger])
    
else :
    print("-----Not Using Data augmentation---------------")
    model.fit(train_images, train_labels,
              batch_size=7,
              epochs=20,
              validation_split= 0.3,
              shuffle=True,callbacks=[lr_reducer, early_stopper, csv_logger])